In [1461]:
from PIL import Image
import os

In [1462]:
INPUT_DIR = "train"
OUTPUT_DIR = "train_resized"
NUM_IMAGES = 5488

In [1463]:
def get_input_image_path(image_id):
    return os.path.join(INPUT_DIR, f"img_{image_id:06d}.jpg")

def get_output_image_path(image_id):
    return os.path.join(OUTPUT_DIR, f"img_{image_id:06d}.jpg")

def open_image(image_path):
    try:
        return Image.open(image_path)
    except IOError:
        print(f"Error opening image: {image_path}")
        return None
    
def save_images(images):
    # Create the output directory if it doesn't exist
    os.makedirs(OUTPUT_DIR, exist_ok=True)


    for image in images:
        # Get the filename from the first image
        filename = image.filename
        image_id = int(filename.split("_")[1].split(".")[0])

        # Save the modified image
        image.save(get_output_image_path(image_id))

def save_images_sbs(original, modified):
    # Create the output directory if it doesn't exist
    os.makedirs(OUTPUT_DIR, exist_ok=True)


    for i, image in enumerate(original):
        # Get the filename from the first image
        filename = image.filename
        image_id = int(filename.split("_")[1].split(".")[0])

        # Put original and modified images side by side
        new_image = Image.new('RGB', (modified[i].width * 2, modified[i].height))
        new_image.paste(image, (0, 0))
        new_image.paste(modified[i], (modified[i].width, 0))

        # Save the modified image
        new_image.save(get_output_image_path(image_id))

In [1464]:
"""
Crop images.
"""

CROP_MARGINS = (5, 5, 5, 5)  # left, top, right, bottom

def crop_images(images):
    modified = []

    for image in images:
        width, height = image.size
        left = CROP_MARGINS[0]
        top = CROP_MARGINS[1]
        right = width - CROP_MARGINS[2]
        bottom = height - CROP_MARGINS[3]

        # Crop the image
        new_image = image.crop((left, top, right, bottom))
        new_image.filename = image.filename
        modified.append(new_image)
    
    return modified

In [1465]:
"""
Convert images to grayscale.
"""

def convert_grayscale(images):
    modified = []

    for image in images:
        # Convert the image to grayscale
        new_image = image.convert("L")
        new_image.filename = image.filename
        modified.append(new_image)
    
    return modified

In [1466]:
"""
Increase contrast
"""
from PIL import ImageEnhance

def increase_contrast(images):
    modified = []

    for image in images:
        # Increase the contrast of the image
        enhancer = ImageEnhance.Contrast(image)
        new_image = enhancer.enhance(1.5)  # Increase contrast by 50%
        new_image.filename = image.filename
        modified.append(new_image)
    
    return modified

In [1467]:
"""
Boost exposure if image is too dark
"""
import numpy as np

def get_average_brightness(image):
    if image.mode != "L":
        image = image.convert("L")
    image_array = np.array(image)
    return image_array.mean()

# def boost_exposure(images):
#     modified = []

#     for image in images:
#         # Only boost exposure if the image is too dark
#         avg_brightness = get_average_brightness(image)

#         if avg_brightness < 100:
#             # Increase the brightness of the image
#             enhancer = ImageEnhance.Brightness(image)
#             new_image = enhancer.enhance(100 / avg_brightness)
#             new_image.filename = image.filename
#             modified.append(new_image)
#         else:
#             # If the image is not too dark, keep it as is
#             new_image = image.copy()
#             new_image.filename = image.filename
#             modified.append(new_image)
    
#     return modified

def boost_exposure(images):
    # boost brightness if the centeral area is too dark
    modified = []
    for image in images:
        # Only boost exposure if the image is too dark
        avg_brightness = get_average_brightness(image)

        if avg_brightness < 100:
            # Increase the brightness of the image
            enhancer = ImageEnhance.Brightness(image)
            new_image = enhancer.enhance(100 / avg_brightness)
            new_image.filename = image.filename
            modified.append(new_image)
        else:
            # If the image is not too dark, keep it as is
            new_image = image.copy()
            new_image.filename = image.filename
            modified.append(new_image)
    return modified

In [1468]:
"""
Decrease exposure if image is too bright
"""

def decrease_exposure(images):
    modified = []

    for image in images:
        # Only boost exposure if the image is too dark
        avg_brightness = get_average_brightness(image)

        if avg_brightness > 100:
            # Increase the brightness of the image
            enhancer = ImageEnhance.Brightness(image)
            new_image = enhancer.enhance(100 / avg_brightness)
            new_image.filename = image.filename
            modified.append(new_image)
        else:
            # If the image is not too dark, keep it as is
            new_image = image.copy()
            new_image.filename = image.filename
            modified.append(new_image)
    
    return modified

In [1469]:
"""
Highlight edges in the image using a filter.
"""

from PIL import ImageFilter
def highlight_edges(images):
    modified = []

    for image in images:
        # Apply an edge enhancement filter to the image
        new_image = image.filter(ImageFilter.EDGE_ENHANCE)
        new_image.filename = image.filename
        modified.append(new_image)
    
    return modified

In [1470]:
"""
Increase saturation
"""

def increase_saturation(images):
    modified = []

    for image in images:
        # Increase the saturation of the image
        enhancer = ImageEnhance.Color(image)
        new_image = enhancer.enhance(1.5)
        new_image.filename = image.filename
        modified.append(new_image)
    
    return modified

In [1471]:
import numpy as np
import colorsys
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.patheffects as path_effects
from PIL import Image
from collections import deque

def classify_chunk_context_aware(chunk_rgb, mean_brightness, std_brightness,
                                 sat_thresh=0.4, val_thresh=0.3, sigma=1):
    chunk = np.array(chunk_rgb) / 255.0
    chunk_brightness = np.mean(np.max(chunk, axis=2))

    if chunk_brightness < mean_brightness - std_brightness:
        return "black"
    elif chunk_brightness > mean_brightness + std_brightness:
        return "white"

    hue_weights = {"red": 0, "blue": 0.0, "yellow": 0.0}
    hue_centres = {"red": [0, 360], "blue": [230], "yellow": [55]}

    for row in chunk_rgb:
        for r, g, b in row:
            h, s, v = colorsys.rgb_to_hsv(r / 255.0, g / 255.0, b / 255.0)
            h *= 360

            for color, centres in hue_centres.items():
                if color == "yellow":
                    local_sat_thresh = 0.2
                    local_val_thresh = 0.2
                    local_sigma = sigma * 1.5
                elif color == "red":
                    local_sat_thresh = 0.2
                    local_val_thresh = 0.2
                    local_sigma = sigma * 1.2
                else:
                    local_sat_thresh = sat_thresh
                    local_val_thresh = val_thresh
                    local_sigma = sigma

                if s < local_sat_thresh or v < local_val_thresh:
                    continue

                for centre in centres:
                    dist = min(abs(h - centre), 360 - abs(h - centre))
                    weight = np.exp(-(dist ** 2) / (2 * local_sigma ** 2)) * s * v
                    hue_weights[color] += weight

    # hue_weights["yellow"] *= 1.2
    # hue_weights["red"] *= 1.1
    # hue_weights["blue"] *= 1.1

    max_color = max(hue_weights, key=hue_weights.get)
    max_weight = hue_weights[max_color]
    total_weight = sum(hue_weights.values())

    if max_weight == 0 or (total_weight > 0 and max_weight / total_weight < 0.5):
        return "other"
    return max_color

def dfs_collect_region(label_grid, start_y, start_x, visited):
    rows, cols = len(label_grid), len(label_grid[0])
    label = label_grid[start_y][start_x]
    stack = [(start_y, start_x)]
    visited[start_y][start_x] = True
    coords = [(start_y, start_x)]

    while stack:
        y, x = stack.pop()
        for dy, dx in [(-1,0), (1,0), (0,-1), (0,1), (-1,-1), (1,1), (-1,1), (1,-1)]:
            ny, nx = y + dy, x + dx
            if (0 <= ny < rows and 0 <= nx < cols and
                not visited[ny][nx] and label_grid[ny][nx] == label):
                visited[ny][nx] = True
                stack.append((ny, nx))
                coords.append((ny, nx))

    size = len(coords)
    cy = sum(y for y, _ in coords) / size
    cx = sum(x for _, x in coords) / size

    return {
        "size": size,
        "label": label,
        "cy": cy,
        "cx": cx,
        "coords": coords
    }

def find_best_sign_colour_region(label_grid, allowed_labels={"red", "blue", "yellow"}, min_touching_size=15):
    rows, cols = len(label_grid), len(label_grid[0])
    visited = [[False] * cols for _ in range(rows)]
    centre_y, centre_x = rows // 2, cols // 2
    centre_chunk = (centre_y, centre_x)
    regions = []

    min_touching_size = 0.2 * (rows * cols)

    for y in range(rows):
        for x in range(cols):
            if not visited[y][x] and label_grid[y][x] in allowed_labels:
                region = dfs_collect_region(label_grid, y, x, visited)
                regions.append(region)

    if not regions:
        return 0, None, []

    # Find all regions that contain the centre
    touching = [r for r in regions if centre_chunk in r["coords"] and r["size"] >= min_touching_size]

    if touching:
        # Choose the largest centre-touching region
        best_region = max(touching, key=lambda r: r["size"])
    else:
        # Fallback: choose largest region overall
        best_region = max(regions, key=lambda r: r["size"])

    return best_region["size"], best_region["label"], best_region["coords"]

import numpy as np
from PIL import Image
import cv2

def mask_inside_convex_hull(image, label_grid):
    # Convert image to NumPy array
    img_np = np.array(image)

    # Get the largest colour group and its coordinates
    _, _, coords = find_best_sign_colour_region(label_grid)
    if not coords:
        return image  # No group found, return original

    # Convert coords to proper format for convex hull
    points = np.array(coords, dtype=np.int32).reshape((-1, 1, 2))

    # Create convex hull from group points
    hull = cv2.convexHull(points)

    # Create a black mask and draw the convex hull filled
    mask = np.zeros((img_np.shape[0], img_np.shape[1]), dtype=np.uint8)
    cv2.drawContours(mask, [hull], -1, 255, cv2.FILLED)

    # Apply mask to image (preserve 3 channels)
    if len(img_np.shape) == 3:
        result_np = cv2.bitwise_and(img_np, img_np, mask=mask)
    else:
        # Grayscale image
        result_np = cv2.bitwise_and(img_np, mask)

    # Convert back to PIL image
    return Image.fromarray(result_np)





# 🖼️ Main overlay plot
def plot_chunk_colours_overlay(images, chunk_size=4):
    label_map = {"red": "R", "blue": "B", "yellow": "Y", "black": "K", "white": "W", "other": "O"}
    colour_map = {
        "red": (1, 0, 0, 0.4),
        "blue": (0, 0, 1, 0.4),
        "yellow": (1, 1, 0, 0.4),
        "black": (0, 0, 0, 0.4),
        "white": (1, 1, 1, 0.4),
        "other": (0.5, 0.5, 0.5, 0.4)
    }

    modified = []

    for image in images:
        img_np = np.array(image.convert("RGB"))
        height, width, _ = img_np.shape
        norm_img = img_np / 255.0
        brightness_map = np.max(norm_img, axis=2)
        mean_brightness = np.mean(brightness_map)
        std_brightness = np.std(brightness_map)

        # Classify chunks
        label_grid = []
        for y in range(0, height, chunk_size):
            row = []
            for x in range(0, width, chunk_size):
                chunk = img_np[y:y+chunk_size, x:x+chunk_size]
                label = classify_chunk_context_aware(chunk, mean_brightness, std_brightness)
                row.append(label)
            label_grid.append(row)

        # Use hybrid scoring
        size, label, coords = find_best_sign_colour_region(label_grid)
        # print(f"{image.filename}Selected region: {label} with size {size}")

        # Plot
        fig, ax = plt.subplots(figsize=(5, 5))
        ax.imshow(img_np)
        rows, cols = len(label_grid), len(label_grid[0])
        coord_set = set(coords)

        new_image = mask_inside_convex_hull(image, label_grid)
        new_image.filename = image.filename
        modified.append(new_image)

        for i in range(rows):
            for j in range(cols):
                y = i * chunk_size
                x = j * chunk_size
                label_here = label_grid[i][j]
                overlay_colour = colour_map[label_here]
                letter = label_map[label_here]

                edgecolor = 'cyan' if (i, j) in coord_set else 'white'
                linewidth = 1.2 if (i, j) in coord_set else 0.5

                rect = patches.Rectangle(
                    (x - 0.5, y - 0.5),
                    chunk_size,
                    chunk_size,
                    linewidth=linewidth,
                    edgecolor=edgecolor,
                    facecolor=overlay_colour
                )
                ax.add_patch(rect)

                ax.text(
                    x + chunk_size / 2 - 0.5,
                    y + chunk_size / 2 - 0.5,
                    letter,
                    color='white',
                    ha='center',
                    va='center',
                    fontsize=6,
                    fontweight='bold',
                    path_effects=[
                        path_effects.Stroke(linewidth=1, foreground='black'),
                        path_effects.Normal()
                    ]
                )

        ax.set_xlim(0, width)
        ax.set_ylim(height, 0)
        ax.axis('off')
        plt.tight_layout()

        plt.title("Context-Aware Dominant Colour Overlay")
        plt.show()

        print(f"{image.filename} selected region: {label} with size {size}")

    return modified

In [1472]:
def match_contour_and_mask(images):
    modified = []
    for image in images:
        # Convert Pillow image to OpenCV format
        image_np = np.array(image.convert("RGB"))
        image_cv = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)

        # Convert to grayscale and threshold
        gray = cv2.cvtColor(image_cv, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

        # Find contours
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if not contours:
            return None  # No contours found

        # Choose the largest contour by area
        largest_contour = max(contours, key=cv2.contourArea)

        # Create a blank black mask
        mask = np.zeros_like(gray)

        # Draw the contour onto the mask in white
        cv2.drawContours(mask, [largest_contour], -1, color=255, thickness=-1)

        # Convert the mask back to Pillow image
        mask_pil = Image.fromarray(mask)
        mask_pil.filename = image.filename
        modified.append(mask_pil)

    return modified

In [1473]:

def resize_images(images):
    modified = []

    for image in images:
        # Resize the image
        width, height = image.size
        new_size = (width * 2, height * 2)
        img_np = np.array(image)
        resized = cv2.resize(img_np, new_size, interpolation=cv2.INTER_CUBIC)
        new_image = Image.fromarray(resized)
        new_image.filename = image.filename
        modified.append(new_image)
    
    return modified

In [1474]:
def sharpen_images(images):
    modified = []

    for image in images:
        # Sharpen the image
        enhancer = ImageEnhance.Sharpness(image)
        new_image = enhancer.enhance(2.0)  # Increase sharpness by 100%
        new_image.filename = image.filename
        modified.append(new_image)
    
    return modified

In [1475]:
def remove_noise(images):
    modified = []

    for image in images:
        # Convert the image to grayscale
        gray_image = image.convert("L")
        # Apply a median filter to remove noise
        new_image = gray_image.filter(ImageFilter.MedianFilter(size=3))
        new_image.filename = image.filename
        modified.append(new_image)
    
    return modified

In [1476]:
from PIL import Image
import numpy as np
from skimage.feature import hog
from skimage import exposure

def get_hog_features(images):
    modified = []

    for image in images:
        gray_image = image.convert("L")
        img_np = np.array(gray_image)

        # Compute HOG features and visualisation
        features, hog_image = hog(img_np, orientations=9, pixels_per_cell=(4, 4),
                                  cells_per_block=(2, 2), visualize=True, block_norm='L2-Hys')

        # Enhance contrast for visualisation
        hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

        new_image = Image.fromarray((hog_image_rescaled * 255).astype(np.uint8))
        new_image.filename = image.filename
        modified.append(new_image)
    
    return modified


In [1477]:
from PIL import Image
import numpy as np
import cv2

from PIL import Image
import numpy as np
import cv2

def boost_red_and_desaturate_others(images):
    modified = []

    for image in images:
        # Convert image to RGB → BGR (OpenCV format)
        img_rgb = np.array(image.convert("RGB"))
        img_bgr = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)
        img_hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV).astype(np.float32)

        h, s, v = cv2.split(img_hsv)

        # Define broader red hue range (OpenCV hue: 0-180)
        # Includes dark red to light red, and orangey red
        red_mask = (
            ((h >= 0) & (h <= 10)) |  # Red to orange-red
            ((h >= 165) & (h <= 180))  # Wraparound red
        ) & (s > 50) & (v > 30)  # Ensure it’s not too grey or dark

        non_red_mask = ~red_mask

        # Boost red saturation and brightness
        s[red_mask] = np.clip(s[red_mask] * 2.0, 0, 255)
        v[red_mask] = np.clip(v[red_mask] * 1.3, 0, 255)

        # Desaturate non-reds heavily
        s[non_red_mask] *= 0.1

        # Recombine and convert back to RGB
        img_hsv_mod = cv2.merge([h, s, v]).astype(np.uint8)
        img_bgr_mod = cv2.cvtColor(img_hsv_mod, cv2.COLOR_HSV2BGR)
        img_rgb_mod = cv2.cvtColor(img_bgr_mod, cv2.COLOR_BGR2RGB)

        new_image = Image.fromarray(img_rgb_mod)
        new_image.filename = image.filename
        modified.append(new_image)

    return modified



In [1478]:
filters = [
    resize_images,
    crop_images,
    boost_exposure,
    # boost_red_and_desaturate_others,
    # get_contours,
    # remove_noise,
    # sharpen_images,
    highlight_edges,
    # get_hog_features,
    # decrease_exposure,
    # increase_contrast,
    # plot_chunk_colours_overlay,
    # match_contour_and_mask,
    # convert_grayscale,
    # increase_saturation,
]

def apply_filters(images):
    for filter_func in filters:
        images = filter_func(images)
    return images

In [1479]:
def main():
    images = []

    # Load images
    for i in range(1, NUM_IMAGES + 1):
        image = open_image(get_input_image_path(i))
        if image:
            images.append(image)
        # if i % 200 == 0:
        #     break

    # Apply filters
    modified = apply_filters(images)
    # save_images(modified)

    original_cropped = crop_images(images)
    save_images_sbs(original_cropped, modified)


if __name__ == "__main__":
    main()